Redox flow battery setup using `rfbmodel`

In [1]:
# Change for your directory 
# Remember to pip install . after each change from the
# rfbmodel directory
#!pip install /home/felipe/RFBmodel/ 
!pip install "C:/Users/jbvba/RFBmodelPC/RFBmodel"


Processing c:\users\jbvba\rfbmodelpc\rfbmodel
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rfbmodel: filename=rfbmodel-0.1-py3-none-any.whl size=6465 sha256=0c0ba0112c673b12cd5248793dfb0ad37c972c66a4df7592f3bccd96b22be886
  Stored in directory: C:\Users\jbvba\AppData\Local\Temp\pip-ephem-wheel-cache-s6qd2fnn\wheels\f5\e2\e6\adaf62f013c25d2ed8d5494892f41e9f9fab1d4b2646ed6396
Successfully built rfbmodel
  Attempting uninstall: rfbmodel
    Found existing installation: rfbmodel 0.1
    Uninstalling rfbmodel-0.1:
      Successfully uninstalled rfbmodel-0.1


  DEPRECATION: Building 'rfbmodel' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rfbmodel'. Discussion can be found at https://github.com/pypa/pip/issues/6334


It is always advisable to check the code in a script before converting it into a class. For example, we will test the `conc` function from the cathode.

Suggestion: use SciPy to solve the non linear system

$$ f(\vec{c}) = \vec{b}$$

Now this is implemented in the cathode class

In [2]:
from rfbmodel.cathodes import Cathode
from rfbmodel.agdl import AGDL
from rfbmodel.data import conductivity_funcV5, conductivity_funcV4

In [ ]:
#Define design parameters for cathode

cath_thickness = 50/1000 #Cathode thickness [m]
cath_conductivity = 4400 #Conductivity of the cathode / sigma [S m^-1]
cath_porosity = 0.94 #Cathode porosity (void fraction)

# Construct a cathode
cath = Cathode(cath_thickness, cath_conductivity, cath_porosity)

# Print initial guess
print("c_guess = ", cath.c)

# Solve system of equations
cath.conc()

# Print solution
print("c = ", cath.c)



c_guess =  [1.   1.   1.   1.   2.   0.01]
c =  [ 1.03000000e+00 -1.07292877e-10  1.02985342e+01  8.42961216e+00
  1.96446103e+00  4.35151124e-01]


In [4]:
cath.ionic_resistance()
cath.cathode_resistance()
cath.total_resistance()

# Print solution
print("resistance of cathode = ", cath.cath_resistance, "Ωm2")
print("resistance of catholyte = ", cath.io_resistance, "Ωm2")
print("Total resistance of cathode and catholyte = ", cath.resistance, "Ωm2")


resistance of cathode =  0.0007731975198179214 Ωm2
resistance of catholyte =  0.0014439487366181896 Ωm2
Total resistance of cathode and catholyte =  0.0005035561270062641 Ωm2


AGDL Class

In [5]:
# Construct a Anode Gas Diffusion Layer
agdl = AGDL()

# Calculate AGDL resistance
agdl.calculate_resistance()

# Print solution
print("AGDL Resistance", agdl.resistance, "Ωm2")

AGDL Resistance 4.972409010471659e-09 Ωm2


ACL Class

In [8]:
# Construct a Anode Catalyst Layer
acl = ACL()

# Calculate ACL resistance
acl.calculate_resistance()

# Print solution
print("ACL Resistance", agdl.resistance, "Ωm2")

NameError: name 'ACL' is not defined